In [1]:
from netCDF4 import Dataset
import iris
import numpy as np
import pylab as plt
import cartopy.crs as ccrs
import cartopy.feature
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
#filename = '/Users/ananth/Downloads/SM_sden_ERA5_01Aug1980-31Jul2021_v01.nc'
#cube = iris.load(filename)

filename = '/Users/ananth/Downloads/SM_snod_ERA5_01Aug1980-31Jul2021_v01.nc'
cube = iris.load(filename)

lat_index = 0
lon_index = 0
snowdepth_index = 0

for i in range(2):
    varname = cube[i].var_name
    if varname == 'snod':
        snowdepth_index = i

snow_depth_org = cube[snowdepth_index].data
snow_depth_org[snow_depth_org<0] = 0
filename4 = '/Users/ananth/Downloads/icemotion_weekly_nh_25km_19781105_19781231_v4.1.nc'
cube4 = iris.load(filename4)

lat_index = 0
lon_index = 0

for i in range(5):
    varname = cube4[i].var_name
    if varname == 'latitude':
        lat_index = i
    elif varname == 'longitude':
        lon_index = i
        
lon = cube4[lon_index].data
lat = cube4[lat_index].data

snow_depth = snow_depth_org[-365:,:,:].data # this is for the one year .. slice 

snow_depth_1yr = snow_depth_org[-30:,:,:].data


/Users/ananth/miniconda3/envs/tensorflow_env/lib/python3.7/site-packages/iris/fileformats/cf.py:649: UserWarning: Missing CF-netCDF grid mapping variable 'lambert_azimuthal_equal_area', referenced by netCDF variable 'snod'
  warnings.warn(message % (name, nc_var_name))


In [ ]:
depth_cube = cube[snowdepth_index]
time_arr = depth_cube.coord('time').points
np.shape(time_arr)

start_date = '08/01/1980'
date_1 = datetime.strptime(start_date,'%m/%d/%Y').date()
date_arr =[start_date]
#new_date = []
new_date = 0

for i in range(len(time_arr)-1):
    new_date = date_1+timedelta(days = i)
    new_date_str = new_date.strftime('%m/%d/%Y')
    date_arr += [new_date_str]
    #print(date_arr)

index_1 = date_arr.index('01/01/2020')
index_2 = date_arr.index('12/31/2020')
#date_arr[index_1:index_2+1]

snow_depth_1yr_slice = snow_depth_org[index_1:index_2+1,:,:].data

start_date_2 = datetime.strptime('01/01/2020','%m/%d/%Y').date()

snow_depth_annual = np.zeros((12,361,361))

for i in range(12):
    
    new_date_2 = start_date_2+relativedelta(months= i) 
    new_date_2_str = new_date_2.strftime('%m/%d/%Y')
    index_2 = date_arr.index(new_date_2_str)
    snow_depth_1month = snow_depth_org[index_1:index_2+1,:,:]
    snow_depth_1month_mean = np.mean(snow_depth_1month,axis=0)
    snow_depth_annual[i,:,:] = snow_depth_1month_mean.data
    
    print(index_1,index_2)
    index_1 = index_2
    print(date_arr[index_2])
    
print(np.shape(snow_depth_annual))

In [ ]:
np.mean(snow_depth_1yr_slice)

In [ ]:
# convert snow_depth_annual from 12,361,361 array to 12,722,722

lat_org = np.sort(lat.flatten())
lon_org = np.sort(lon.flatten())


lat_1d = lat.flatten()
lon_1d = lon.flatten()

lat_sort_index = np.argsort(lat_1d)
lon_sort_index = np.argsort(lon_1d)



lat_lon = np.zeros((722,722))

new_SD = np.zeros((12,722,722))

for i in range(722):
    index1 = np.where(lat_sort_index == i)
    index2 = np.where(lon_sort_index == i)
    
    new_SD[0,index1,index2] = snow_depth_annual[0,] 
    


"""
for i in range(len(lat_org)):
    index1 = np.where(lat_1d[i]==lat_org)
    index2 = np.where(lon_1d[index_1] == lon_org)
    for j in range(len(index1)):
        new_SD[0,j,index2]
        
        for j in range(len(lon_org)):
            index3 = np.where(lon)
        

np.shape(snow_depth_annual)

"""

a = np.array([1,2,3,4,5,0])
b= np.array([12,1,14,3,7,0])
print(np.argsort(a))
print(np.argsort(b))

np.where(lat_sort_index==1)
lat_sort_index[4]


In [ ]:
# save snow depth, latitude and longitude into a netcdf file
import netCDF4 as nc
import os

fn = 'snow_depth_1.nc'

os.remove(fn)
ds = nc.Dataset(fn,'w',format = 'NETCDF4')

time = ds.createDimension('time',12)
lat_dim = ds.createDimension('unknown1',361)
lon_dim = ds.createDimension('unknown2',361)

temp_time = np.arange(1,13,1)
temp_lat = np.linspace(1,2,361)
temp_lon = np.linspace(1,2,361)
np.shape(temp_time)

times = ds.createVariable('time', 'f4', ('time',))
lats = ds.createVariable('unknown1', 'f4', ('unknown1',))
lons = ds.createVariable('unknown2', 'f4', ('unknown2',))
value = ds.createVariable('snow_depth', 'f4', ('time', 'unknown1', 'unknown2',))
value.units = 'm'

lats[:]=temp_lat
lons[:]=temp_lon
times[:]= temp_time

value[:,:,:] = snow_depth_annual
ds.close()

cube111 = iris.load('snow_depth_1.nc')
cube111


In [ ]:
#save latitude into a cube

import netCDF4 as nc
import os

fn = 'latitude.nc'

os.remove(fn)
ds = nc.Dataset(fn,'w',format = 'NETCDF4')

#time = ds.createDimension('time',12)
lat_dim = ds.createDimension('unknown1',361)
lon_dim = ds.createDimension('unknown2',361)

#temp_time = np.arange(1,13,1)
temp_lat = np.linspace(1,2,361)
temp_lon = np.linspace(1,2,361)
#np.shape(temp_time)

#times = ds.createVariable('time', 'f4', ('time',))
lats = ds.createVariable('unknown1', 'f4', ('unknown1',))
lons = ds.createVariable('unknown2', 'f4', ('unknown2',))
value = ds.createVariable('latitude', 'f4', ('unknown1', 'unknown2',))
value.units = 'm'

lats[:]=temp_lat
lons[:]=temp_lon
#times[:]= temp_time

value[:,:] = lat
ds.close()

cube111 = iris.load('latitude.nc')
cube111


In [ ]:
#save longitude array into a cube

import netCDF4 as nc
import os

fn = 'longitude.nc'

#os.remove(fn)
ds = nc.Dataset(fn,'w',format = 'NETCDF4')

#time = ds.createDimension('time',12)
lat_dim = ds.createDimension('unknown1',361)
lon_dim = ds.createDimension('unknown2',361)

#temp_time = np.arange(1,13,1)
temp_lat = np.linspace(1,2,361)
temp_lon = np.linspace(1,2,361)
#np.shape(temp_time)

#times = ds.createVariable('time', 'f4', ('time',))
lats = ds.createVariable('unknown1', 'f4', ('unknown1',))
lons = ds.createVariable('unknown2', 'f4', ('unknown2',))
value = ds.createVariable('longitude', 'f4', ('unknown1', 'unknown2',))
value.units = 'm'

lats[:]=temp_lat
lons[:]=temp_lon
#times[:]= temp_time

value[:,:] = lon
ds.close()

cube111 = iris.load('longitude.nc')
cube111


In [ ]:
snow_depth_1yr = snow_depth_org[-730:-365,:,:].data
snow_depth_1yr[snow_depth_1yr<0] = 0
snow_depth = np.mean(snow_depth_1yr,axis=0)
np.max(snow_depth)

#np.max(snow_depth_annual)


In [ ]:


fig, ax = plt.subplots(1,1,figsize=(8,8),  subplot_kw={'projection': ccrs.NorthPolarStereo()})

ax.add_feature(cartopy.feature.LAND, edgecolor='black',zorder=1)
#ax.add_feature(cartopy.feature.OCEAN)

lat_1d = lat.flatten();
lon_1d = lon.flatten();

snowdepth_1d = snow_depth.flatten()*100

#j = np.isnan(snowdepth_1d)
#s_depth_slice = snowdepth_1d[~j]
#lat_slice = lat_1d[~j]
#lon_slice = lon_1d[~j]

#colors = snowdepth_1d/np.max(snowdepth_1d_depth_slice)

colors = snowdepth_1d

im = ax.scatter(lon_1d, lat_1d,c = colors,s = 100, marker = '.',transform = ccrs.PlateCarree(),zorder = 5)
cb = fig.colorbar(im,ax=ax,fraction=0.046, pad=0.04)
ax.set_extent([-180, 180, 90, 60], ccrs.PlateCarree())
cb.ax.tick_params(labelsize=18)
plt.title('Snow depth (cm) (SnowModel LG data)',fontsize = 24)
ax.gridlines()

"""         
#ax.scatter(lat[])
#ax.colorbar()
ax.set_title('Snow depth in the Arctic (cm)',fontsize=25)
cb.ax.tick_params(labelsize=18)
#ax.legend(fontsize='x-large')
ax.gridlines()
"""

In [ ]:
np.shape(snow_depth)

In [ ]:
filename3 = '/Users/ananth/Downloads/SM_sden_MERRA2_01Aug1980-31Jul2021_v01.nc'
cube3 = iris.load(filename3)


In [ ]:
filename4 = '/Users/ananth/Downloads/icemotion_weekly_nh_25km_19781105_19781231_v4.1.nc'
cube4 = iris.load(filename4)
cube4

In [ ]:
#start_date = '08-01-1980'
#date_1 = datetime.strptime(start_date,'%m-%d-%Y').date()

start_date = '08/01/1980'
date_1 = datetime.strptime(start_date,'%m/%d/%Y').date()
#x = datetime.datetime.strptime('08/01/1980','%m/%d/%y')

In [ ]:
from datetime import datetime,timedelta

datetime_str = '09/19/22'

datetime_object = datetime.strptime(datetime_str, '%m/%d/%y')

In [ ]:
index_1 = date_arr.index('01/01/2020')
index_2 = date_arr.index('12/31/2020')
date_arr[index_1:index_2+1]

In [ ]:
#tomcat lat and lon

tomcat_lat = [87.86, 85.10, 82.31, 79.53, 76.74,
        73.95, 71.16, 68.37, 65.58, 62.79,
        60.00, 57.21, 54.42, 51.63, 48.84,
        46.04, 43.25, 40.46, 37.67, 34.88,
        32.09, 29.30, 26.51, 23.72, 20.93,
        18.14, 15.35, 12.56,  9.77,  6.98,
         4.19,  1.40, -1.40, -4.19, -6.98,
        -9.77,-12.56,-15.35,-18.14,-20.93,
       -23.72,-26.51,-29.30,-32.09,-34.88,
       -37.67,-40.46,-43.25,-46.04,-48.84,
       -51.63,-54.42,-57.21,-60.00,-62.79,
       -65.58,-68.37,-71.16,-73.95,-76.74,
       -79.53,-82.31,-85.10,-87.86]

tomcat_lon_360 = np.arange(0,360,2.8125)

tomcat_lon = []
new_lon = 0

for i in range(len(tomcat_lon_360)):
    if tomcat_lon_360[i] > 180:
        new_lon = tomcat_lon_360[i]-360
    else:
        new_lon = tomcat_lon_360[i]
    tomcat_lon = np.append(tomcat_lon,new_lon)
    
tomcat_lon = np.sort(tomcat_lon)

In [ ]:
print('min and max lat = ', np.min(lat_1d),np.max(lat_1d))
print('min and max lon = ', np.min(lon_1d),np.max(lon_1d))
X,Y = np.meshgrid (lon_1d,lat_1d)
for i in range(len(tomcat_lat)):
    for j in range(len(tomcat_lon)):
        flag = (tomcat_lat[i]>np.min(lat_1d) and tomcat_lat[i]<np.max(lat_1d) and tomcat_lon[j]>np.min(lon_1d) and tomcat_lon[j]<np.max(lon_1d))
        if flag == True:
            print(flag)
            lat_point = tomcat_lat[i]
            lon_point = tomcat_lon[j]
            data = snow_depth_annual[0,:,:]
            
            
    

In [ ]:
snow_depth_annual

In [ ]:
np.shape(X)

In [ ]:
a = np.array([12,12,13,4,5,6])
np.where(a==4)

In [ ]:
lat_org = np.sort(lat.flatten())
lon_org = np.sort(lon.flatten())

lat_1d = lat.flatten();
lon_1d = lon.flatten();

np.where(lat_org[10]==lat_1d)

#lat_org[0]


In [ ]:
lat_org[0]

In [ ]:
lat_org[360]

In [ ]:
lat_1d[360]

In [ ]:
for i in range(361):
    print('lon_original index = ', i)
    for j in range(361):
        if j!=i:
            flag = (lon[i,:] - lon[j,:] == 0)
    #print('value of flag = ', flag)
    #print('shape of flag = ', np.shape(flag))
            if flag.all() == True:
            #print('hello')
                print ('same data at other indices = ', j)

In [ ]:

for i in range(361):
    print('lat_original index = ', i)
    for j in range(361):
        if j!=i:
            flag = (lat[i,:] - lat[j,:] == 0)
    #print('value of flag = ', flag)
    #print('shape of flag = ', np.shape(flag))
            if flag.all() == True:
            #print('hello')
                print ('same data at other indices = ', j)

In [ ]:
lat[2,:]

In [ ]:
lat_1d = lat.flatten();
lon_1d = lon.flatten();

snowdepth_1d = snow_depth.flatten()*100

In [ ]:
snowdepth_1d.reshape((12,361,361))

In [ ]:
np.shape(snow_depth_annual)

In [ ]:
import numpy as np
temp_lon = np.linspace(1,2,361)
temp_lon.shape
